In [30]:
import cv2
import numpy as np
import csv
import face_recognition
import pandas as pd
import os
from datetime import datetime
import smtplib
import ssl #secure socket layer
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import tkinter
#libraries for gui
from tkinter import *
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from tkinter import ttk
import tkinter as tk
from PIL import ImageTk, Image

In [31]:
path = 'images'
images = []
studentNames = []
myList = os.listdir(path)
print(myList)
for cu_img in myList:
    current_Img = cv2.imread(f'{path}/{cu_img}')
    images.append(current_Img)
    studentNames.append(os.path.splitext(cu_img)[0])
print(studentNames)

['amit.jpg', 'mrunal.jpg', 'Prasad.jpg', 'prem.jpg', 'sanchita.jpg', 'shfaque.jpg', 'vijay.jpg']
['amit', 'mrunal', 'Prasad', 'prem', 'sanchita', 'shfaque', 'vijay']


In [40]:
def faceEncodings(images):
    encodeList = []
    for img in images:
        img =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
encodeListK = faceEncodings(images)
print(encodeListK)

[array([-0.09099687,  0.12977123, -0.00186227, -0.02046251, -0.02096293,
       -0.0202143 , -0.02260322, -0.03684559,  0.24832518, -0.13413095,
        0.19103135, -0.01279408, -0.1391094 , -0.09320188, -0.0669741 ,
        0.10764645, -0.14912128, -0.14202015, -0.06683253, -0.14006472,
        0.02966917,  0.04480225, -0.04760804,  0.00223907, -0.22521801,
       -0.33125335, -0.12583499, -0.12325618,  0.04764462, -0.10656697,
        0.00835004,  0.08359715, -0.17404015, -0.06929576,  0.02644281,
        0.056615  , -0.05861757,  0.02786714,  0.12466998,  0.03997519,
       -0.07784159, -0.02108696,  0.04824921,  0.33681244,  0.18262853,
        0.05920459, -0.00313449, -0.02370472,  0.08976468, -0.20717207,
        0.04484188,  0.10157218,  0.08370047,  0.13588455,  0.11426616,
       -0.11605068, -0.01489084,  0.06743024, -0.1677032 ,  0.0706985 ,
       -0.03892891, -0.08929631, -0.07638887,  0.02741285,  0.20257215,
        0.09389956, -0.07255454, -0.1351409 ,  0.19008985, -0.1

In [41]:
def markAttendence(name):
    with open('finaledi.csv','r+') as f:
        mydataList=f.readlines()
        nameList=[]
        for line in mydataList:
            entry=line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now=datetime.now()
                ttString=now.strftime('%H-%M-%S')
                dtString=now.strftime('%d-%m-%Y')
                f.writelines(f'\n{name},{dtString},{ttString}')
                break
  

In [44]:
print("1.take attendence\n2.registration\n3.send attendence")
userinput=input()        
if userinput=='1':
    faceEncodings(images)

    count=0
    while count<1:
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        faces = cv2.resize(frame, (0, 0), None, 0.25, 0.25)
        faces = cv2.cvtColor(faces, cv2.COLOR_BGR2RGB)

        facesCurrentFrame = face_recognition.face_locations(faces)
        encodesCurrentFrame = face_recognition.face_encodings(faces, facesCurrentFrame)
        for encodeFace, faceLoc in zip(encodesCurrentFrame, facesCurrentFrame):
            matches = face_recognition.compare_faces(encodeListK, encodeFace)
            faceDis = face_recognition.face_distance(encodeListK, encodeFace)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = studentNames[matchIndex].upper()#converting small case letters to uppercase
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0,0), 2)
                cv2.imshow('Webcam', frame)
                k=cv2.waitKey(5000)#display a window for given milliseconds
                markAttendence(name)
                print("Attendence Marked")
                count=count+1
                break
                
            else :
                print("Unkonwn")
                break
        cap.release()
        cv2.destroyAllWindows()


elif userinput=='2':
    cap=cv2.VideoCapture(0)
    print("enter your name")
    name=input()
    print("registration is in progress....")
    while (True):
        ret,img=cap.read()
        cv2.imshow("camera",img)
    
        if not ret:
            break
        k=cv2.waitKey(1)
        if k%256==32:          #for space key
            print("regestration completed")
            file="F:/3rd sem/myproject/images/"+name+'.jpg'
            cv2.imwrite(file,img)
            break
    cap.release()
    cv2.destroyAllWindows()
  
elif userinput=='3':
    smtp_port=587

    smtp_server="smtp.gmail.com"

    email_from="shejoleprem01@gmail.com"
    email_to="premshejole04@gmail.com"
    pswd="ifakquyxmvcpvupl"
    now=datetime.now()
    curr_date=now.strftime('%d-%m-%Y')
    subject="Attendence "+curr_date 

    msg=MIMEMultipart()
    msg['From']=email_from
    msg['to']=email_to
    msg['subject']=subject

    body="hello sir\nmail from smart attendence system\n ATTENDANCE SUCCEFULLY MARKED!"
    msg.attach(MIMEText(body,'plain'))
    filename="finaledi.csv"
    attachment=open(filename,'rb')

    attachment_package=MIMEBase('application','octet-stream')
    attachment_package.set_payload((attachment).read())
    encoders.encode_base64(attachment_package)
    attachment_package.add_header('content-disposition',"attachment;filename= " +filename)

    msg.attach(attachment_package)
    text=msg.as_string()
    TIE_server=smtplib.SMTP(smtp_server,smtp_port)
    TIE_server.starttls()
    TIE_server.login(email_from,pswd)
    TIE_server.sendmail(email_from,email_to,text)
    print("Attendence sent")


1.take attendence
2.registration
3.send attendence
3
Attendence sent


In [ ]:
1
